In [3]:
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests

exchange_urls = {
    "Wazirx": "https://api.wazirx.com/api/v2/tickers",
    "CoinDCX": "https://api.coindcx.com/exchange/ticker",
}

column_names_wazirx = ["Base Unit", "Quote Unit", "Sell", "Buy", "Name"]

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/pawan/Downloads/creds.json', scope)
gc = gspread.authorize(credentials)

sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1k9vZJsj8vSzxu7r7rQyrcBj-2Cnw5lTNPVgbY59Sr3I/edit?pli=1#gid=0').sheet1


def fetch_exchange_data(exchange_name, url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        exchange_data = response.json()
        return exchange_data
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data from {exchange_name}: {e}")
        return None


def update_wazirx_sheet():
    try:
        sheet.clear()
        row_count = 0
        for exchange_name, exchange_url in exchange_urls.items():
            if exchange_name != "Wazirx":
                continue

            sheet.append_row([exchange_name])  
            sheet.append_row(column_names_wazirx) 

            exchange_data = fetch_exchange_data(exchange_name, exchange_url)
            if exchange_data:
                sorted_data = sorted(exchange_data.items(), key=lambda x: float(x[1].get('buy', 0)), reverse=True)[:20]
                for key, value in sorted_data:
                    row_data = [value.get('base_unit', ''), value.get('quote_unit', ''),
                                value.get('sell', ''), value.get('buy', ''),
                                value.get('name', '')]
                    sheet.append_row(row_data)
                    row_count += 1
                    if row_count >= 20:
                        break

    except KeyboardInterrupt:
        print("Process interrupted. Exiting...")


def update_coindcx_sheet():
    try:
        row_count = 0
        for exchange_name, exchange_url in exchange_urls.items():
            if exchange_name != "CoinDCX":
                continue

            exchange_data = fetch_exchange_data(exchange_name, exchange_url)
            if exchange_data:
                sheet.append_row([])
                sheet.append_row([exchange_name]) 

                sorted_data = sorted(exchange_data, key=lambda x: float(x.get('ask', 0)), reverse=True)[:20]
                for market_data in sorted_data:
                    if market_data.get('market', '') == "BTCINR_insta":
                        continue

                    row_data = [market_data.get('market', ''), market_data.get('change_24_hour', ''),
                                market_data.get('last_price', '')]
                    sheet.append_row(row_data)
                    row_count += 1
                    if row_count >= 20:
                        break

    except KeyboardInterrupt:
        print("Process interrupted. Exiting...")


def main():
    try:
        sheet.clear()  
        while True:
            update_wazirx_sheet()
            time.sleep(5) 

            update_coindcx_sheet()
            time.sleep(55)  

    except KeyboardInterrupt:
        print("Process interrupted. Exiting...")


if __name__ == "__main__":
    main()


Process interrupted. Exiting...
